Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [6]:
import numpy as np
from scipy import stats

z = (9.57 - 9.5) / (0.4 / np.sqrt(160))
round(2 * stats.norm.sf(z), 4)

0.0269

---

Имеются данные о стоимости и размерах 53940 бриллиантов: diamonds.txt

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На обучающей выборке настройте две регрессионные модели:

Линейную регрессию с помощью LinearRegression без параметров
случайный лес из 10 деревьев с помощью RandomForestRegressor с random_state=1.
Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости $\alpha=0.05?$

In [9]:
import pandas as pd

data = pd.read_csv('diamonds.txt', sep='\t')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   carat   53940 non-null  float64
 1   depth   53940 non-null  float64
 2   table   53940 non-null  float64
 3   price   53940 non-null  int64  
 4   x       53940 non-null  float64
 5   y       53940 non-null  float64
 6   z       53940 non-null  float64
dtypes: float64(6), int64(1)
memory usage: 2.9 MB


,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [50]:
from sklearn import linear_model, ensemble, model_selection

X = data.drop('price', axis=1).to_numpy()
y = data['price'].to_numpy()
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=1)

reg = linear_model.LinearRegression().fit(X_train, y_train)
# Выборка абсолютных отклонений для линейной регрессии
X1 = abs(y_test - reg.predict(X_test))

rfr = ensemble.RandomForestRegressor(n_estimators=10, random_state=1).fit(X_train, y_train)
# Выборка абсолютных отклонений для случайного леса
X2 = abs(y_test - rfr.predict(X_test))

In [51]:
stats.ttest_rel(X1, X2)

Ttest_relResult(statistic=13.01772978388187, pvalue=1.6551745750705154e-38)

---

В предыдущей задаче посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до десятков (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).

In [70]:
from statsmodels.stats.weightstats import *

int(DescrStatsW(X1 - X2).tconfint_mean()[0]*0.1)*10

70